# Part 2/4 to generate Aachen dataset:
## (Binary images with LiDAR footprints and tilt distribution)
## Convert pickle to imagery

In [1]:
from pathlib import Path
import numpy as np
from shapely.geometry.polygon import Polygon
import pickle
from PIL import Image, ImageDraw
from math import ceil
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
aachen_dataset_path = Path('F:/datasets/aachen')
aachen_model_path = Path('output/aachen_rooftops.pkl')

output_tiles_path = Path('F:/datasets/aachen_model')
output_tiles_path_img = output_tiles_path / 'image'
output_tiles_path_label = output_tiles_path / 'label'

output_tiles_path_img.mkdir(parents=True, exist_ok=True)
output_tiles_path_label.mkdir(parents=True, exist_ok=True)

#### Load pickle file generated in previous notebook

In [3]:
with open(aachen_model_path, 'rb') as f:
        buildings = pickle.load(f)

#### Utility functions

In [4]:
def get_img_buildings(polygon_list, center_latlon, size, level):
    center_lat, center_lon = center_latlon
    size_x, size_y = size

    center_x, center_y = ConversionHelper.latlon_toXY(center_lat, center_lon, level)

    offset_x = center_x - int(size_x/2)
    offset_y = center_y - int(size_y/2)

    lat1, lon1 = ConversionHelper.xy_tolatlon(offset_x, offset_y, level)
    lat2, lon2 = ConversionHelper.xy_tolatlon(offset_x + size_x, offset_y + size_y, level)

    min_lat, max_lat = min(lat1, lat2), max(lat1, lat2)
    min_lon, max_lon = min(lon1, lon2), max(lon1, lon2)

    filtered_polygon_list = []
    for sublist in polygon_list:
        for sl in sublist:
            if any((item[0] > min_lat and item[0] < max_lat) and (item[1] > min_lon and item[1] < max_lon) for item in sl):
                filtered_polygon_list.append(sublist)
    
    return filtered_polygon_list

def get_center(filename, correct_watermark=False):

    location, img_center_lat, img_center_lon = filename.replace('_', '.').split('-')
    
    img_center_lat, img_center_lon = float(img_center_lat), float(img_center_lon)

    if correct_watermark:
        IMG_WATERMARK_X = 0
        IMG_WATERMARK_Y = 25
        IMG_LEVEL = 19

        x_tmp, y_tmp = ConversionHelper.latlon_toXY(img_center_lat, img_center_lon, IMG_LEVEL)

        x_tmp -= int(IMG_WATERMARK_X / 2)
        y_tmp -= int(IMG_WATERMARK_Y / 2)

        img_center_lat, img_center_lon = ConversionHelper.xy_tolatlon(x_tmp, y_tmp, IMG_LEVEL)

    return img_center_lat, img_center_lon

def polygon_latlon_toxy(footprints, size, center_latlon, level):
    size_x, size_y = size
    center_lat, center_lon = center_latlon

    center_x, center_y = ConversionHelper.latlon_toXY(center_lat, center_lon, level)

    offset_x = center_x - int(size_x/2)
    offset_y = center_y - int(size_y/2)
    
    footprints_xy = []

    for footprint in footprints:
        geoms = []
        for geom in footprint:
            coords = []
            for coord in geom:
                x_coord, y_coord = ConversionHelper.latlon_toXY(coord[0], coord[1], level)
                
                coords.append((x_coord - offset_x, y_coord - offset_y))
            geoms.append(coords)
        footprints_xy.append(geoms)
    
    return footprints_xy

def polygons_to_bin(polygons, size):
    size_x, size_y = size

    img_bin = Image.new('L', (size_x, size_y))
    draw = ImageDraw.Draw(img_bin, 'L')
    for polygon in polygons:
        draw.polygon(polygon[0], fill=255, outline=None)
    
    return img_bin


from math import cos, sin, pi, log, atan, exp, floor
class ConversionHelper:
    '''
    Bing Maps conversion helper functions as specified in:
    https://docs.microsoft.com/en-us/bingmaps/articles/bing-maps-tile-system
    '''
    EARTH_CIRCUMFERENCE = 6378137
    MINLAT = -85.05112878
    MAXLAT = 85.05112878
    MINLON = -180
    MAXLON = 180
    MAXLVL = 22
    
    def map_width(lvl):
        '''width in pixels of the world map at a given level of detail'''
        return 256 << lvl
    def ground_resolution(lat, lvl):
        '''distance on the ground that’s represented by a single pixel in the map'''
        return cos(lat * pi / 180) * 2 * pi * ConversionHelper.EARTH_CIRCUMFERENCE / ConversionHelper.map_width(lvl)
    def map_scale(lat, lvl, screen_dpi):
        '''ratio between map distance and ground distance'''
        return 0.0254 / (ConversionHelper.ground_resolution(lat, lvl) * screen_dpi)

    def latlon_toXY(lat, lon, lvl):
        '''converts a point from latlon WGS-84 in degrees into the pixel representation in the world map'''
        lat, lon = max(ConversionHelper.MINLAT, min(ConversionHelper.MAXLAT, lat)), \
                    max(ConversionHelper.MINLON, min(ConversionHelper.MAXLON, lon))
        
        sin_lat = sin(lat * pi/180)
        pixelX = ((lon + 180) / 360) * ConversionHelper.map_width(lvl)
        pixelY = (0.5 - log((1 + sin_lat) / (1 - sin_lat)) / (4 * pi)) * ConversionHelper.map_width(lvl)
        
        return floor(pixelX + 0.5), floor(pixelY + 0.5)

    def xy_tolatlon(pixelX, pixelY, lvl):
        '''converts a point from the pixel representation in the world map into latlon WGS-84 in degrees'''
        mapSize = ConversionHelper.map_width(lvl);  
        x = (pixelX / mapSize) - 0.5
        y = 0.5 - pixelY / mapSize

        latitude = 90 - 360 * atan(exp(-y * 2 * pi)) / pi  
        longitude = 360 * x
        
        return(latitude, longitude)

    def xy_totile(pixelX, pixelY):
        '''converts a point in pixel coordinates to the coordinates of the tile containing it'''
        return(floor(pixelX / 256), floor(pixelY / 256))

    def tile_toxy(pixelX, pixelY):
        '''converts the coordinates of a tile into the pixel coordinates of its upper-left corner'''
        return(pixelX * 256, pixelY * 256)

In [5]:
def get_tile_center(position, tile_size, n_tiles_x, n_tiles_y, img_center_latlon, size, level):
    center_lat, center_lon = img_center_latlon
    size_x, size_y = size
    
    center_x, center_y = ConversionHelper.latlon_toXY(center_lat, center_lon, level)
    
    offset_x = center_x - int(size_x/2)
    offset_y = center_y - int(size_y/2)
    
    tile_center_x = ((position % n_tiles_x) * tile_size) + offset_x + (tile_size // 2)
    tile_center_y = ((position // n_tiles_x) * tile_size)  + offset_y + (tile_size // 2)
    
    tile_center_lat, tile_center_lon = ConversionHelper.xy_tolatlon(tile_center_x, tile_center_y, level)
    
    return tile_center_lat, tile_center_lon

#### Generate images in level 19 and with a pixel size of 512x512

In [6]:
LEVEL = 19
TILE_SIZE = 512
count = 0
tile_count = 1
for image_path in aachen_dataset_path.glob('*.png'):
    img = Image.open(image_path)
    image = np.asarray(img)
    
    img_center_lat, img_center_lon = get_center(image_path.stem, correct_watermark=True)
    img_size_y, img_size_x, img_channels = image.shape
    
    img_buildings_latlon = get_img_buildings(buildings, (img_center_lat, img_center_lon), (img_size_x, img_size_y), LEVEL)
    
    
    if img_buildings_latlon:
        # THERE IS A BUILDING IN THE IMAGE, DIVIDE TILES
        print(image_path)
        count += 1
        print('Processing image ',count)
        
        tiles = []
        for h in range(0,img_size_y,TILE_SIZE):
            for w in range(0,img_size_x,TILE_SIZE):
                i = image[h:h+TILE_SIZE,w:w+TILE_SIZE]
                i = np.pad(i, ((0, TILE_SIZE - i.shape[0]), (0, TILE_SIZE - i.shape[1]), (0, 0)),
                        mode='constant', constant_values=0)
                tiles.append(i)
        tiles = np.array(tiles)
        
        n_tiles_x = ceil(img_size_x / TILE_SIZE)
        n_tiles_y = ceil(img_size_y / TILE_SIZE)
        for i, tile in enumerate(tiles):
            tile_center_lat, tile_center_lon = get_tile_center(i, TILE_SIZE, n_tiles_x, n_tiles_y, 
                                                               (img_center_lat, img_center_lon), 
                                                               (img_size_x, img_size_y), LEVEL)
            
            tile_buildings_latlon = get_img_buildings(img_buildings_latlon, (tile_center_lat, tile_center_lon), (TILE_SIZE, TILE_SIZE), LEVEL)
            
            if tile_buildings_latlon:
                tile_buildings_xy = polygon_latlon_toxy(tile_buildings_latlon,
                                                        (TILE_SIZE, TILE_SIZE),
                                                        (tile_center_lat, tile_center_lon),
                                                        LEVEL)
                label = np.asarray(polygons_to_bin(tile_buildings_xy, (TILE_SIZE, TILE_SIZE)))
                
                cv2.imwrite(str(output_tiles_path_img / str(tile_count)) + '.png', tile)
                cv2.imwrite(str(output_tiles_path_label / str(tile_count)) + '.png', label)
                
                tile_count += 1

F:\datasets\aachen\aachen-50_682098986727134-6_146523356437683.png
Processing image  1
F:\datasets\aachen\aachen-50_682098986727134-6_173345446586609.png
Processing image  2
F:\datasets\aachen\aachen-50_682098986727134-6_178709864616394.png
Processing image  3
F:\datasets\aachen\aachen-50_682098986727134-6_184074282646179.png
Processing image  4
F:\datasets\aachen\aachen-50_68460569449507-6_146523356437683.png
Processing image  5
F:\datasets\aachen\aachen-50_68460569449507-6_151887774467468.png
Processing image  6
F:\datasets\aachen\aachen-50_68460569449507-6_167981028556824.png
Processing image  7
F:\datasets\aachen\aachen-50_68460569449507-6_173345446586609.png
Processing image  8
F:\datasets\aachen\aachen-50_687112268353964-6_141158938407898.png
Processing image  9
F:\datasets\aachen\aachen-50_687112268353964-6_146523356437683.png
Processing image  10
F:\datasets\aachen\aachen-50_687112268353964-6_151887774467468.png
Processing image  11
F:\datasets\aachen\aachen-50_687112268353964-

F:\datasets\aachen\aachen-50_709665407566746-6_1948031187057495.png
Processing image  95
F:\datasets\aachen\aachen-50_709665407566746-6_200167536735535.png
Processing image  96
F:\datasets\aachen\aachen-50_709665407566746-6_20553195476532.png
Processing image  97
F:\datasets\aachen\aachen-50_71217064246507-6_1250656843185425.png
Processing image  98
F:\datasets\aachen\aachen-50_71217064246507-6_130430102348328.png
Processing image  99
F:\datasets\aachen\aachen-50_71217064246507-6_135794520378113.png
Processing image  100
F:\datasets\aachen\aachen-50_71217064246507-6_141158938407898.png
Processing image  101
F:\datasets\aachen\aachen-50_71217064246507-6_146523356437683.png
Processing image  102
F:\datasets\aachen\aachen-50_71217064246507-6_162616610527039.png
Processing image  103
F:\datasets\aachen\aachen-50_71217064246507-6_173345446586609.png
Processing image  104
F:\datasets\aachen\aachen-50_71217064246507-6_178709864616394.png
Processing image  105
F:\datasets\aachen\aachen-50_7121

F:\datasets\aachen\aachen-50_7271992403936-6_060692667961121.png
Processing image  188
F:\datasets\aachen\aachen-50_7271992403936-6_071421504020691.png
Processing image  189
F:\datasets\aachen\aachen-50_7271992403936-6_082150340080261.png
Processing image  190
F:\datasets\aachen\aachen-50_7271992403936-6_087514758110046.png
Processing image  191
F:\datasets\aachen\aachen-50_7271992403936-6_108972430229187.png
Processing image  192
F:\datasets\aachen\aachen-50_7271992403936-6_1250656843185425.png
Processing image  193
F:\datasets\aachen\aachen-50_7271992403936-6_130430102348328.png
Processing image  194
F:\datasets\aachen\aachen-50_7271992403936-6_135794520378113.png
Processing image  195
F:\datasets\aachen\aachen-50_7271992403936-6_141158938407898.png
Processing image  196
F:\datasets\aachen\aachen-50_7271992403936-6_146523356437683.png
Processing image  197
F:\datasets\aachen\aachen-50_7271992403936-6_151887774467468.png
Processing image  198
F:\datasets\aachen\aachen-50_7271992403936

F:\datasets\aachen\aachen-50_73971939058565-6_0928791761398315.png
Processing image  282
F:\datasets\aachen\aachen-50_73971939058565-6_098243594169617.png
Processing image  283
F:\datasets\aachen\aachen-50_73971939058565-6_108972430229187.png
Processing image  284
F:\datasets\aachen\aachen-50_73971939058565-6_130430102348328.png
Processing image  285
F:\datasets\aachen\aachen-50_73971939058565-6_146523356437683.png
Processing image  286
F:\datasets\aachen\aachen-50_73971939058565-6_151887774467468.png
Processing image  287
F:\datasets\aachen\aachen-50_73971939058565-6_157252192497253.png
Processing image  288
F:\datasets\aachen\aachen-50_73971939058565-6_162616610527039.png
Processing image  289
F:\datasets\aachen\aachen-50_73971939058565-6_178709864616394.png
Processing image  290
F:\datasets\aachen\aachen-50_73971939058565-6_184074282646179.png
Processing image  291
F:\datasets\aachen\aachen-50_73971939058565-6_189438700675964.png
Processing image  292
F:\datasets\aachen\aachen-50_74

F:\datasets\aachen\aachen-50_74973310129934-6_130430102348328.png
Processing image  376
F:\datasets\aachen\aachen-50_74973310129934-6_135794520378113.png
Processing image  377
F:\datasets\aachen\aachen-50_74973310129934-6_141158938407898.png
Processing image  378
F:\datasets\aachen\aachen-50_74973310129934-6_146523356437683.png
Processing image  379
F:\datasets\aachen\aachen-50_74973310129934-6_151887774467468.png
Processing image  380
F:\datasets\aachen\aachen-50_74973310129934-6_157252192497253.png
Processing image  381
F:\datasets\aachen\aachen-50_74973310129934-6_162616610527039.png
Processing image  382
F:\datasets\aachen\aachen-50_74973310129934-6_167981028556824.png
Processing image  383
F:\datasets\aachen\aachen-50_74973310129934-6_173345446586609.png
Processing image  384
F:\datasets\aachen\aachen-50_74973310129934-6_178709864616394.png
Processing image  385
F:\datasets\aachen\aachen-50_74973310129934-6_184074282646179.png
Processing image  386
F:\datasets\aachen\aachen-50_752

...

F:\datasets\aachen\aachen-50_77475800909394-6_119701266288757.png
Processing image  655
F:\datasets\aachen\aachen-50_77475800909394-6_1250656843185425.png
Processing image  656
F:\datasets\aachen\aachen-50_77475800909394-6_130430102348328.png
Processing image  657
F:\datasets\aachen\aachen-50_77475800909394-6_135794520378113.png
Processing image  658
F:\datasets\aachen\aachen-50_77475800909394-6_141158938407898.png
Processing image  659
F:\datasets\aachen\aachen-50_77475800909394-6_146523356437683.png
Processing image  660
F:\datasets\aachen\aachen-50_77475800909394-6_151887774467468.png
Processing image  661
F:\datasets\aachen\aachen-50_77475800909394-6_157252192497253.png
Processing image  662
F:\datasets\aachen\aachen-50_77475800909394-6_162616610527039.png
Processing image  663
F:\datasets\aachen\aachen-50_77475800909394-6_167981028556824.png
Processing image  664
F:\datasets\aachen\aachen-50_77475800909394-6_173345446586609.png
Processing image  665
F:\datasets\aachen\aachen-50_77

F:\datasets\aachen\aachen-50_782262871762875-6_141158938407898.png
Processing image  748
F:\datasets\aachen\aachen-50_782262871762875-6_146523356437683.png
Processing image  749
F:\datasets\aachen\aachen-50_782262871762875-6_151887774467468.png
Processing image  750
F:\datasets\aachen\aachen-50_782262871762875-6_157252192497253.png
Processing image  751
F:\datasets\aachen\aachen-50_782262871762875-6_162616610527039.png
Processing image  752
F:\datasets\aachen\aachen-50_782262871762875-6_167981028556824.png
Processing image  753
F:\datasets\aachen\aachen-50_782262871762875-6_173345446586609.png
Processing image  754
F:\datasets\aachen\aachen-50_782262871762875-6_178709864616394.png
Processing image  755
F:\datasets\aachen\aachen-50_78476422501639-6_001684069633484.png
Processing image  756
F:\datasets\aachen\aachen-50_78476422501639-6_02850615978241.png
Processing image  757
F:\datasets\aachen\aachen-50_78476422501639-6_033870577812195.png
Processing image  758
F:\datasets\aachen\aachen

F:\datasets\aachen\aachen-50_792267481902066-6_03923499584198.png
Processing image  842
F:\datasets\aachen\aachen-50_792267481902066-6_04996383190155.png
Processing image  843
F:\datasets\aachen\aachen-50_792267481902066-6_0553282499313354.png
Processing image  844
F:\datasets\aachen\aachen-50_792267481902066-6_060692667961121.png
Processing image  845
F:\datasets\aachen\aachen-50_792267481902066-6_066057085990906.png
Processing image  846
F:\datasets\aachen\aachen-50_792267481902066-6_071421504020691.png
Processing image  847
F:\datasets\aachen\aachen-50_792267481902066-6_076785922050476.png
Processing image  848
F:\datasets\aachen\aachen-50_792267481902066-6_082150340080261.png
Processing image  849
F:\datasets\aachen\aachen-50_792267481902066-6_087514758110046.png
Processing image  850
F:\datasets\aachen\aachen-50_792267481902066-6_0928791761398315.png
Processing image  851
F:\datasets\aachen\aachen-50_792267481902066-6_098243594169617.png
Processing image  852
F:\datasets\aachen\aa

F:\datasets\aachen\aachen-50_79976953452569-6_087514758110046.png
Processing image  935
F:\datasets\aachen\aachen-50_79976953452569-6_0928791761398315.png
Processing image  936
F:\datasets\aachen\aachen-50_79976953452569-6_098243594169617.png
Processing image  937
F:\datasets\aachen\aachen-50_79976953452569-6_103608012199402.png
Processing image  938
F:\datasets\aachen\aachen-50_79976953452569-6_108972430229187.png
Processing image  939
F:\datasets\aachen\aachen-50_79976953452569-6_114336848258972.png
Processing image  940
F:\datasets\aachen\aachen-50_79976953452569-6_119701266288757.png
Processing image  941
F:\datasets\aachen\aachen-50_79976953452569-6_1250656843185425.png
Processing image  942
F:\datasets\aachen\aachen-50_79976953452569-6_130430102348328.png
Processing image  943
F:\datasets\aachen\aachen-50_79976953452569-6_135794520378113.png
Processing image  944
F:\datasets\aachen\aachen-50_79976953452569-6_141158938407898.png
Processing image  945
F:\datasets\aachen\aachen-50_7

F:\datasets\aachen\aachen-50_809770398172454-6_103608012199402.png
Processing image  1028
F:\datasets\aachen\aachen-50_81227027960286-6_02850615978241.png
Processing image  1029
F:\datasets\aachen\aachen-50_81227027960286-6_033870577812195.png
Processing image  1030
F:\datasets\aachen\aachen-50_81227027960286-6_03923499584198.png
Processing image  1031
F:\datasets\aachen\aachen-50_81227027960286-6_044599413871765.png
Processing image  1032
F:\datasets\aachen\aachen-50_81227027960286-6_04996383190155.png
Processing image  1033
F:\datasets\aachen\aachen-50_81227027960286-6_0553282499313354.png
Processing image  1034
F:\datasets\aachen\aachen-50_81227027960286-6_060692667961121.png
Processing image  1035
F:\datasets\aachen\aachen-50_81227027960286-6_066057085990906.png
Processing image  1036
F:\datasets\aachen\aachen-50_81227027960286-6_071421504020691.png
Processing image  1037
F:\datasets\aachen\aachen-50_81227027960286-6_087514758110046.png
Processing image  1038
F:\datasets\aachen\aac